In [ ]:
!pip install Pillow

**DataSet Properties**

In [ ]:
import os
import time

# Define the directory containing the images
directory = '/content/drive/MyDrive/Suspected labeled dataset/ILLegal'

# Get the list of all files in the directory
all_files = os.listdir(directory)
total_files = len(all_files)

# Function to display progress
def display_progress(progress):
    print(f'\rProgress: [{progress * "#"}{(100-progress) * "-"}] {progress}%', end='')

# Rename all files in the directory
for i, filename in enumerate(all_files):
    old_path = os.path.join(directory, filename)
    if os.path.isfile(old_path):
        new_filename = f'Illegal Action {i+1}.jpg'
        new_path = os.path.join(directory, new_filename)
        os.rename(old_path, new_path)
        # Calculate and display progress
        progress = int(((i+1) / total_files) * 100)
        display_progress(progress)
        time.sleep(0.1)  # Adding a slight delay for visibility of progress (can be removed)

print("\nRenaming completed.")


Progress: [####################################################################################################] 100%
Renaming completed.


In [ ]:
from google.colab import drive
import os
from PIL import Image

# Mount Google Drive
drive.mount('/content/drive')

# Set the dataset path
dataset_path = '/content/drive/MyDrive/Suspected bowling action Dataset'

def get_image_properties(image_path):
    with Image.open(image_path) as img:
        return {
            'format': img.format,
            'mode': img.mode,
            'size': img.size
        }

def explore_dataset(path):
    dataset_info = {
        'total_folders': 0,
        'total_files': 0,
        'image_files': 0,
        'non_image_files': 0,
        'image_properties': []
    }

    for root, dirs, files in os.walk(path):
        dataset_info['total_folders'] += len(dirs)
        dataset_info['total_files'] += len(files)
        for file in files:
            file_path = os.path.join(root, file)
            try:
                properties = get_image_properties(file_path)
                dataset_info['image_files'] += 1
                dataset_info['image_properties'].append({
                    'file_path': file_path,
                    'properties': properties
                })
            except Exception as e:
                dataset_info['non_image_files'] += 1

    return dataset_info

# Explore the dataset
dataset_info = explore_dataset(dataset_path)

# Print the dataset info
print(f"Total Folders: {dataset_info['total_folders']}")
print(f"Total Files: {dataset_info['total_files']}")
print(f"Image Files: {dataset_info['image_files']}")
print(f"Non-Image Files: {dataset_info['non_image_files']}")

# Display a few image properties
for img_info in dataset_info['image_properties'][:5]:
    print(f"File: {img_info['file_path']}")
    print(f"Properties: {img_info['properties']}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total Folders: 4
Total Files: 1826
Image Files: 1826
Non-Image Files: 0
File: /content/drive/MyDrive/Suspected bowling action Dataset/Left Arm Legal/IMG_7231.jpg
Properties: {'format': 'JPEG', 'mode': 'RGB', 'size': (3024, 4032)}
File: /content/drive/MyDrive/Suspected bowling action Dataset/Left Arm Legal/IMG_7230.jpg
Properties: {'format': 'JPEG', 'mode': 'RGB', 'size': (3024, 4032)}
File: /content/drive/MyDrive/Suspected bowling action Dataset/Left Arm Legal/IMG_7234.jpg
Properties: {'format': 'JPEG', 'mode': 'RGB', 'size': (3024, 4032)}
File: /content/drive/MyDrive/Suspected bowling action Dataset/Left Arm Legal/IMG_7229.jpg
Properties: {'format': 'JPEG', 'mode': 'RGB', 'size': (3024, 4032)}
File: /content/drive/MyDrive/Suspected bowling action Dataset/Left Arm Legal/IMG_7227.jpg
Properties: {'format': 'JPEG', 'mode': 'RGB', 'size': (3024, 4032)}


**Model Code**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
#... (rest of the code)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json

# Define the paths
dataset_path = '/content/drive/MyDrive/Suspected labeled dataset'
save_path = '/content/drive/MyDrive/Model'  # Path to save model and metadata

# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% validation data
)

# Load images from the directories
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # VGG16 expects 224x224 images
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

# Print the number of samples in the training and validation sets
print(f"Number of training samples: {train_generator.samples}")
print(f"Number of validation samples: {validation_generator.samples}")

# Ensure the number of steps per epoch is greater than zero
steps_per_epoch = max(1, train_generator.samples // train_generator.batch_size)
validation_steps = max(1, validation_generator.samples // validation_generator.batch_size)

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of VGG16
for layer in base_model.layers[:15]:
    layer.trainable = False

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Adjust the output layer according to the number of classes
])

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=1e-5),
    metrics=['accuracy']
)

# Summary of the model
model.summary()

# Define callbacks
checkpoint = ModelCheckpoint(f'{save_path}/best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    epochs=10,
    callbacks=[checkpoint, early_stopping]
)

# Save the model architecture and weights
model.save(f'{save_path}/Suspected_BA_final_model.h5')

# Save model metadata
metadata = {
    'input_shape': (224, 224, 3),
    'num_classes': train_generator.num_classes,
    'base_model': 'VGG16',
    'trainable_layers': 15,
    'optimizer': 'RMSprop',
    'learning_rate': 1e-5,
    'loss_function': 'categorical_crossentropy',
    'epochs': 10,
    'callbacks': ['ModelCheckpoint', 'EarlyStopping']
}

with open(f'{save_path}/Suspected_BA_model_metadata.json', 'w') as f:
    json.dump(metadata, f)


Found 1406 images belonging to 2 classes.
Found 351 images belonging to 2 classes.
Number of training samples: 1406
Number of validation samples: 351
Error in model summary: Undefined shapes are not supported.


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=/content/drive/MyDrive/Model/best_model.h5

**Testing**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import json
import numpy as np
import os

# Load the model
model_path = '/content/drive/MyDrive/Model/Suspected_BA_final_model.h5'
model = load_model(model_path)

# Load the model metadata
metadata_path = '/content/drive/MyDrive/Model/Suspected_BA_model_metadata.json'
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Define the class labels based on your training setup
class_labels = ['Illegal', 'Legal']  # Adjust if necessary

# Function to preprocess the image
def preprocess_image(image_path, target_size):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# Function to make a prediction
def predict_image(image_path):
    img_array = preprocess_image(image_path, (224, 224))  # Adjust target size based on your model
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    predicted_label = class_labels[predicted_class[0]]
    return predicted_label

# Test the model with a sample image
test_image_path = '/content/drive/MyDrive/Testing data/Ill2.jpg'  # Replace with the path to your test image
predicted_label = predict_image(test_image_path)
print(f'The action in the given image is: {predicted_label}')


1/1 [==============================] - 0s 242ms/step
The action in the given image is: Illegal


In [ ]:
# prompt: google drive mount code

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/Model/Suspected_BA_final_model.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
# prompt: give me the path of the saved file

model_path = 'model.tflite'
